In [ ]:
import sagemaker
import boto3
from sagemaker.estimator import Estimator
from sagemaker.tensorflow import TensorFlow
from sagemaker.local import LocalSession
from sagemaker import get_execution_role

In [ ]:
role = get_execution_role()
session = LocalSession()
session.config = {'local': {'local_code': True}}

In [ ]:
sm_training_s3_input_location="<S3 location to the processing job artifacts>" #for local testing, this variable could be a directory in the local file
sm_training_s3_output_location="<s3 location for training output>" #for local testing, this variable could be a directory in the local file
epochs = 1 # change the number of epochs for the job

In [ ]:
metric_definitions=[
       {'Name': 'train:error', 'Regex': 'loss: (.*?) -'},
       {'Name': 'validation:acc', 'Regex': 'acc: (.*)'}
]

hyperparams = {
                'save_weights' : '/opt/ml/model/sm_model', 
                'dataDir' : '/opt/ml/input/data/training', 
                'epochs' :  epochs, 
                'tol' : 50
            }

In [ ]:
estimator = TensorFlow(
    framework_version="2.5.1",
    role=role,
    instance_count=1,
    instance_type='local_gpu',
    volume_size=225,
    output_path=sm_training_s3_output_location,
    sagemaker_session=session,
    hyperparameters=hyperparams,
    metric_definitions=metric_definitions,
    source_dir="../src/training",
    entry_point='train.py',
)

In [ ]:
estimator.fit({'training': sm_training_s3_input_location})